In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import kagglehub

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load point cloud dataset from Drive
X_train_pc = np.load('/content/drive/MyDrive/Deep2025/ModelNet40-simple-pc/MN40_X_train_pc.npy')
y_train_pc = np.load('/content/drive/MyDrive/Deep2025/ModelNet40-simple-pc/MN40_y_train_pc.npy')
X_test_pc = np.load('/content/drive/MyDrive/Deep2025/ModelNet40-simple-pc/MN40_X_test_pc.npy')
y_test_pc = np.load('/content/drive/MyDrive/Deep2025/ModelNet40-simple-pc/MN40_y_test_pc.npy')

print("✅ Loaded point cloud data:")
print("🔹 X_train_pc:", X_train_pc.shape)
print("🔹 y_train_pc:", y_train_pc.shape)
print("🔹 X_test_pc:", X_test_pc.shape)
print("🔹 y_test_pc:", y_test_pc.shape)

Mounted at /content/drive
✅ Loaded point cloud data:
🔹 X_train_pc: (9843, 1024, 3)
🔹 y_train_pc: (9843,)
🔹 X_test_pc: (2468, 1024, 3)
🔹 y_test_pc: (2468,)


In [ ]:
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense, Dropout, BatchNormalization, Activation

num_classes = len(np.unique(y_train_pc))

input_points = Input(shape=(1024, 3))

# MLP network (shared weights)
x = Conv1D(64, 1, padding='valid')(input_points)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv1D(128, 1, padding='valid')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = Conv1D(1024, 1, padding='valid')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Global feature
x = GlobalMaxPooling1D()(x)

# Fully connected classification head
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

# Define model
model = Model(inputs=input_points, outputs=output)

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Summary (optional)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1024, 3)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 1024, 64)       │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 1024, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 1024, 128)      │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 1024, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 1024, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 1024, 1024)     │       132,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1024, 1024)     │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 1024, 1024)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 1024)           │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 40)             │        10,280 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 811,944 (3.10 MB)

 Trainable params: 809,512 (3.09 MB)

 Non-trainable params: 2,432 (9.50 KB)

In [ ]:
history = model.fit(
    X_train_pc, y_train_pc,
    validation_data=(X_test_pc, y_test_pc),
    epochs=40,
    batch_size=32,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True),
        tf.keras.callbacks.ReduceLROnPlateau(patience=3, factor=0.5, min_lr=1e-5, verbose=1)
    ]
)

Epoch 1/40
308/308 ━━━━━━━━━━━━━━━━━━━━ 18s 32ms/step - accuracy: 0.1210 - loss: 3.8286 - val_accuracy: 0.0405 - val_loss: 3.8120 - learning_rate: 0.0010
Epoch 2/40
308/308 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.1841 - loss: 3.0539 - val_accuracy: 0.0405 - val_loss: 4.1904 - learning_rate: 0.0010
Epoch 3/40
308/308 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.2231 - loss: 2.8811 - val_accuracy: 0.0405 - val_loss: 4.5024 - learning_rate: 0.0010
Epoch 4/40
308/308 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.2431 - loss: 2.7271
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
308/308 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.2431 - loss: 2.7271 - val_accuracy: 0.0405 - val_loss: 4.8587 - learning_rate: 0.0010
Epoch 5/40
308/308 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.2804 - loss: 2.5941 - val_accuracy: 0.0405 - val_loss: 4.8026 - learning_rate: 5.0000e-04
Epoch 6/40
308/308 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.2818 - los